# Create Data

In [6]:
%load_ext sql
%sql sqlite:///ecommerce_V2.db

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [7]:
import pandas as pd
import sqlite3
# Connect to DB
conn = sqlite3.connect("ecommerce_V2.db")

In [8]:
%%sql

-- Products table
CREATE TABLE products (
    product_id INTEGER PRIMARY KEY,
    product_name TEXT,
    category TEXT,
    price REAL
);

-- Customers table
CREATE TABLE customers (
    customer_id INTEGER PRIMARY KEY,
    customer_name TEXT,
    region TEXT
);

-- Orders table
CREATE TABLE orders (
    order_id INTEGER PRIMARY KEY,
    customer_id INTEGER,
    order_date DATE,
    FOREIGN KEY(customer_id) REFERENCES customers(customer_id)
);

-- Order Items table
CREATE TABLE order_items (
    order_item_id INTEGER PRIMARY KEY,
    order_id INTEGER,
    product_id INTEGER,
    quantity INTEGER,
    FOREIGN KEY(order_id) REFERENCES orders(order_id),
    FOREIGN KEY(product_id) REFERENCES products(product_id)
);

   sqlite://
 * sqlite:///ecommerce_V2.db
Done.
Done.
Done.
Done.


[]

In [9]:
%%sql

INSERT INTO products (product_id, product_name, category, price) VALUES
(1, 'Laptop', 'Electronics', 1200),
(2, 'Mouse', 'Electronics', 25),
(3, 'Notebook', 'Stationery', 5),
(4, 'Pen', 'Stationery', 2),
(5, 'Office Chair', 'Furniture', 150),
(6, 'Desk', 'Furniture', 300),
(7, 'Headphones', 'Electronics', 80),
(8, 'Water Bottle', 'Accessories', 15),
(9, 'Battery', 'Electronics', 10),
(10, 'Monitor', 'Electronics', 200);


INSERT INTO customers (customer_id, customer_name, region) VALUES
(1, 'Alice', 'US'),
(2, 'Bob', 'US'),
(3, 'Charlie', 'India'),
(4, 'Diana', 'Germany'),
(5, 'Eve', 'India');


INSERT INTO orders (order_id, customer_id, order_date) VALUES
(101, 1, '2023-01-15'),
(102, 2, '2023-01-17'),
(103, 1, '2023-02-10'),
(104, 3, '2023-03-05'),
(105, 4, '2023-04-22'),
(106, 1, '2023-04-30'),
(107, 5, '2023-06-01'),
(108, 3, '2023-07-20'),
(109, 2, '2023-07-25');


INSERT INTO order_items (order_item_id, order_id, product_id, quantity) VALUES
(1001, 101, 1, 1),     -- Alice buys 1 Laptop
(1002, 101, 2, 2),     -- Alice buys 2 Mice
(1003, 102, 5, 1),     -- Bob buys 1 Office Chair
(1004, 103, 3, 10),    -- Alice buys 10 Notebooks
(1005, 103, 4, 5),     -- Alice buys 5 Pens
(1006, 104, 7, 1),     -- Charlie buys 1 Headphones
(1007, 104, 9, 4),     -- Charlie buys 4 Batteries
(1008, 105, 6, 1),     -- Diana buys 1 Desk
(1009, 106, 8, 2),     -- Alice buys 2 Water Bottles
(1010, 107, 9, 1),     -- Eve buys 1 Battery
(1011, 108, 4, 3),     -- Charlie buys 3 Pens
(1012, 109, 10, 1);    -- Bob buys 1 Monitor


   sqlite://
 * sqlite:///ecommerce_V2.db
10 rows affected.
5 rows affected.
9 rows affected.
12 rows affected.


[]

# Questions

In [ ]:
# Find the top 2 most expensive products per category.

In [18]:
query = """
with ranked_products as(
SELECT 
    category,
    product_id, 
    price,
    DENSE_RANK() OVER(PARTITION BY category ORDER BY price DESC) as rank
FROM products
ORDER BY 1 ASC, 3 DESC)

select * from ranked_products where rank <= 2
"""

df = pd.read_sql_query(query, conn)
df

,category,product_id,price,rank
0,Accessories,8,15.0,1
1,Electronics,1,1200.0,1
2,Electronics,10,200.0,2
3,Furniture,6,300.0,1
4,Furniture,5,150.0,2
5,Stationery,3,5.0,1
6,Stationery,4,2.0,2


In [19]:
# For each customer, calculate the number of distinct months in which they placed an order after their first purchase month.

In [26]:
query = """
with monthly_orders as(
SELECT 
    customer_id,
    strftime('%Y-%m', order_date) AS month,
    order_id
FROM orders
)

select 
    customer_id,
    count(distinct month) - 1 as num_unique_months
from monthly_orders
group by 1
"""

df = pd.read_sql_query(query, conn)
df

,customer_id,num_unique_months
0,1,2
1,2,1
2,3,1
3,4,0
4,5,0


In [27]:
# For each customer, show their monthly spend, and a 3-month rolling sum of their spend.

In [37]:
query = """ 
WITH monthly_spends AS (
SELECT
    customer_id,
    strftime('%Y-%m', order_date) as month,
    sum(price * quantity) as spend
FROM
    order_items a
LEFT JOIN
    orders b ON a.order_id = b.order_id
LEFT JOIN
    products c ON a.product_id = c.product_id
GROUP BY
    1,2)

select
    *,
    sum(spend) OVER (PARTITION BY customer_id ORDER BY month ASC ROWS BETWEEN 2 PRECEDING AND CURRENT ROW) AS rolling_3month_sum
from
    monthly_spends
"""

df = pd.read_sql_query(query, conn)
df

,customer_id,month,spend,rolling_3month_sum
0,1,2023-01,1250.0,1250.0
1,1,2023-02,60.0,1310.0
2,1,2023-04,30.0,1340.0
3,2,2023-01,150.0,150.0
4,2,2023-07,200.0,350.0
5,3,2023-03,120.0,120.0
6,3,2023-07,6.0,126.0
7,4,2023-04,300.0,300.0
8,5,2023-06,10.0,10.0


In [ ]:
# Write a query to find all products that have never been ordered.

In [46]:
query = """ 
WITH product_purchases AS (
SELECT
    product_id,
    count(distinct order_id) as num_orders
FROM
    order_items
GROUP BY
    1)

select
    a.*,
    b.num_orders
from products a
left join product_purchases b
on a.product_id = b.product_id
where num_orders IS NULL
"""

df = pd.read_sql_query(query, conn)
df

,product_id,product_name,category,price,num_orders


In [47]:
query = """ 
SELECT *
FROM products p
LEFT JOIN order_items oi ON p.product_id = oi.product_id
WHERE oi.order_id IS NULL;
"""

df = pd.read_sql_query(query, conn)
df

,product_id,product_name,category,price,order_item_id,order_id,product_id,quantity


In [ ]:
# For each month, compute the 90th percentile of total order value.

In [65]:
query = """ 
WITH order_totals AS (
  SELECT 
    a.order_id,
    strftime('%Y-%m', b.order_date) AS order_month,
    SUM(c.price * a.quantity) AS order_value
  FROM order_items a
  JOIN orders b ON a.order_id = b.order_id
  JOIN products c ON a.product_id = c.product_id
  GROUP BY a.order_id
),


ranked_orders AS (
  SELECT
    order_month,
    order_value,
    ROW_NUMBER() OVER (
      PARTITION BY order_month 
      ORDER BY order_value
    ) AS rn,
    COUNT(*) OVER (
      PARTITION BY order_month
    ) AS total_orders
  FROM order_totals
),


percentiles AS (
  SELECT
    *,
    CAST(rn AS FLOAT) / total_orders AS percentile
  FROM ranked_orders
)


SELECT
  order_month,
  MAX(order_value) AS p90_order_value
FROM percentiles
WHERE percentile >= 0.9
GROUP BY order_month;
"""

df = pd.read_sql_query(query, conn)
df

,order_month,p90_order_value
0,2023-01,1250.0
1,2023-02,60.0
2,2023-03,120.0
3,2023-04,300.0
4,2023-06,10.0
5,2023-07,200.0


In [66]:
# Categorize customers into segments based on their total lifetime spend:

# High (> $500)
# Medium ($200–500)
# Low (< $200)

In [71]:
query = """ 
WITH spends AS (
SELECT
    customer_id,
    sum(price * quantity) as spend
FROM
    order_items a
LEFT JOIN
    orders b ON a.order_id = b.order_id
LEFT JOIN
    products c ON a.product_id = c.product_id
GROUP BY
    1)

SELECT
    *,
    CASE 
        WHEN spend >= 500 THEN 'High'
        WHEN spend < 500 AND spend >= 200 THEN 'Medium'
        WHEN spend < 200 THEN 'Low' 
        ELSE 'Unknown'
    END AS spend_category
FROM
    spends
"""

df = pd.read_sql_query(query, conn)
df

,customer_id,spend,spend_category
0,1,1340.0,High
1,2,350.0,Medium
2,3,126.0,Low
3,4,300.0,Medium
4,5,10.0,Low


In [ ]:
# Find all orders which include only one unique product and that product's name is "battery".

In [79]:
query = """ 
WITH orders_products AS (
SELECT
    *,
    COUNT(order_item_id) OVER(PARTITION BY order_id) as num_items
FROM
    order_items a
LEFT JOIN
    products c ON a.product_id = c.product_id
GROUP BY
    1)

SELECT
    *
FROM
    orders_products
WHERE
    num_items = 1 and product_name = 'Battery'
"""

df = pd.read_sql_query(query, conn)
df

,order_item_id,order_id,product_id,quantity,product_id:1,product_name,category,price,num_items
0,1010,107,9,1,9,Battery,Electronics,10.0,1


In [ ]:
# Find customers who have placed orders that include products from at least 3 different categories.

In [90]:
query = """ 
WITH order_category_counts AS (
  SELECT
    o.order_id,
    b.customer_id,
    COUNT(DISTINCT p.category) AS num_categories
  FROM order_items o
  JOIN products p ON o.product_id = p.product_id
  JOIN orders b ON o.order_id = b.order_id
  GROUP BY o.order_id
)

SELECT DISTINCT customer_id
FROM order_category_counts
WHERE num_categories >= 3;
"""

df = pd.read_sql_query(query, conn)
df

,customer_id


In [ ]:
# For each product category, compute the change in rank of its monthly total sales compared to the previous month.

In [92]:
query = """ 
WITH category_monthly_revenue AS (
  SELECT
    strftime('%Y-%m', o.order_date) AS order_month,
    p.category,
    SUM(p.price * oi.quantity) AS revenue
  FROM order_items oi
  JOIN products p ON oi.product_id = p.product_id
  JOIN orders o ON oi.order_id = o.order_id
  GROUP BY order_month, p.category
),


category_ranks AS (
  SELECT
    *,
    RANK() OVER (PARTITION BY order_month ORDER BY revenue DESC) AS monthly_rank
  FROM category_monthly_revenue
),


rank_changes AS (
  SELECT
    category,
    order_month,
    monthly_rank,
    LAG(monthly_rank) OVER (PARTITION BY category ORDER BY order_month) AS previous_month_rank
  FROM category_ranks
)


SELECT
  category,
  order_month,
  monthly_rank,
  previous_month_rank,
  ABS(monthly_rank - previous_month_rank) AS rank_volatility
FROM rank_changes
WHERE previous_month_rank IS NOT NULL;
"""

df = pd.read_sql_query(query, conn)
df

,category,order_month,monthly_rank,previous_month_rank,rank_volatility
0,Electronics,2023-03,1,1,0
1,Electronics,2023-06,1,1,0
2,Electronics,2023-07,1,1,0
3,Furniture,2023-04,1,2,1
4,Stationery,2023-07,2,1,1
